In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.dataset as dataset
import lib.params as params
import lib.network as network
import lib.path as path
import lib.utils as utils
from datetime import datetime

In [4]:
# read params
with open("config/train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))

    
data_all=np.array(sorted(path.construct_path_lists("out","data_")))
label_all=np.array(sorted(path.construct_path_lists("out","labels_")))
# print(data_all.shape,label_all.shape)

print("initiating network...")
net=network.R2N2(learning_rate)
print(" ...network initiated")

training with a learning rate of 0.1 for 1 epochs with batchs of size 3
initiating network...
 ...network initiated


In [5]:
loss=np.load("/Users/micmelesse/Documents/thesis/aws/model_0.1_5_10 2018-01-30_17:07:54/epoch_003/loss.npy")

In [7]:
i=7
sample_data=np.load(data_all[i])
sample_label=np.load(label_all[i])
utils.imsave_sequence(sample_data[:,:,:,0],"seq_{}.png".format(i))

In [ ]:
model_dir="out/model_{}_{}_{} {}".format(learning_rate,epoch,batch_size,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
    
# train network
loss_all=[]
acc_all=[]
N=len(data_all)
print("starting training")
for e in range(epoch):
    start_time=time.time()
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    
    loss_epoch=[]
    acc_epoch=[]
    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        data=utils.npy_stack(data)
        label=utils.npy_stack(label)
        fd={net.X:data, net.Y: label}
        train_step=net.train_step(fd)
        loss_epoch.append(net.batch_loss.eval(fd))
        print("epoch_{:03d}-batch_{:03d}: loss={}".format(e,batch_number,loss_epoch[-1]))
        batch_number+=1
    save_dir="{}/epoch_{:03d}".format(model_dir,e)
    loss_all.append(loss_epoch)
    net.save(save_dir,"loss",loss_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))